# Classification multi-étiquette

In [1]:
# Chargement des modules necessaires

import os

import random

import numpy as np

import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# On declare les chemins vers les donnees :

Image = 'Data'
        
train_data_dir = 'Data/TRAIN'
validation_data_dir = 'Data/VALIDATION'
test_data_dir = 'Data/TEST'

# Dimmension et path :
img_width, img_height = 224, 224

ImageTRAINCOVID = os.listdir(Image + '/TRAIN/COVID')
ImageTRAINATELECTASIS = os.listdir(Image + '/TRAIN/ATELECTASIS')
ImageTRAINNORMAL = os.listdir(Image + '/TRAIN/NORMAL')
ImageTRAINEFFUSION = os.listdir(Image + '/TRAIN/EFFUSION')
ImageTRAINEMPHYSEMA = os.listdir(Image + '/TRAIN/EMPHYSEMA')
ImageTRAININFILTRATION = os.listdir(Image + '/TRAIN/INFILTRATION')
ImageTRAINPNEUMONIE = os.listdir(Image + '/TRAIN/PNEUMONIE')

ImageVALIDATIONCOVID = os.listdir(Image + '/VALIDATION/COVID')
ImageVALIDATIONATELECTASIS = os.listdir(Image + '/VALIDATION/ATELECTASIS')
ImageVALIDATIONNORMAL = os.listdir(Image + '/VALIDATION/NORMAL')
ImageVALIDATIONEFFUSION = os.listdir(Image + '/VALIDATION/EFFUSION')
ImageVALIDATIONEMPHYSEMA = os.listdir(Image + '/VALIDATION/EMPHYSEMA')
ImageVALIDATIONINFILTRATION = os.listdir(Image + '/VALIDATION/INFILTRATION')
ImageVALIDATIONPNEUMONIE = os.listdir(Image + '/VALIDATION/PNEUMONIE')

ImageTESTCOVID = os.listdir(Image + '/TEST/COVID')
ImageTESTATELECTASIS = os.listdir(Image + '/TEST/ATELECTASIS')
ImageTESTNORMAL = os.listdir(Image + '/TEST/NORMAL')
ImageTESTEFFUSION = os.listdir(Image + '/TEST/EFFUSION')
ImageTESTEMPHYSEMA = os.listdir(Image + '/TEST/EMPHYSEMA')
ImageTESTINFILTRATION = os.listdir(Image + '/TEST/INFILTRATION')
ImageTESTPNEUMONIE = os.listdir(Image + '/TEST/PNEUMONIE')

print('Il y a ' + str(len(ImageTRAINCOVID)) + ' images d\'entrainement de patient covid.') 
print('Il y a ' + str(len(ImageTRAINNORMAL)) + ' images d\'entrainement de patient sain.') 
print('Il y a ' + str(len(ImageTRAINATELECTASIS)) + ' images d\'entrainement de patient atelectasis.') 
print('Il y a ' + str(len(ImageTRAINEFFUSION)) + ' images d\'entrainement de patient effusion.') 
print('Il y a ' + str(len(ImageTRAINEMPHYSEMA)) + ' images d\'entrainement de patient emphyseme.') 
print('Il y a ' + str(len(ImageTRAININFILTRATION)) + ' images d\'entrainement de patient infltration.')
print('Il y a ' + str(len(ImageTRAINPNEUMONIE)) + ' images d\'entrainement de patient pneumonie.')

print('Il y a ' + str(len(ImageVALIDATIONCOVID)) + ' images de validation de patient covid.') 
print('Il y a ' + str(len(ImageVALIDATIONNORMAL)) + ' images de validation de patient sain.') 
print('Il y a ' + str(len(ImageVALIDATIONATELECTASIS)) + ' images de validation de patient atelectasis.') 
print('Il y a ' + str(len(ImageVALIDATIONEFFUSION)) + ' images de validation de patient effusion.') 
print('Il y a ' + str(len(ImageVALIDATIONEMPHYSEMA)) + ' images de validation de patient emphyseme.') 
print('Il y a ' + str(len(ImageVALIDATIONINFILTRATION)) + ' images de validation de patient infltration.')
print('Il y a ' + str(len(ImageVALIDATIONPNEUMONIE)) + ' images de validation de patient pneumonie.')

print('Il y a ' + str(len(ImageTESTCOVID)) + ' images test de patient covid.') 
print('Il y a ' + str(len(ImageTESTNORMAL)) + ' images test de patient sain.')
print('Il y a ' + str(len(ImageTESTATELECTASIS)) + ' images test de patient atelectasis.') 
print('Il y a ' + str(len(ImageTESTEFFUSION)) + ' images test de patient effusion.') 
print('Il y a ' + str(len(ImageTESTEMPHYSEMA)) + ' images test de patient emphyseme.') 
print('Il y a ' + str(len(ImageTESTINFILTRATION)) + ' images test de patient infltration.')
print('Il y a ' + str(len(ImageTESTPNEUMONIE)) + ' images test de patient pneumonie.')

Il y a 1386 images d'entrainement de patient covid.
Il y a 1106 images d'entrainement de patient sain.
Il y a 800 images d'entrainement de patient atelectasis.
Il y a 800 images d'entrainement de patient effusion.
Il y a 1380 images d'entrainement de patient emphyseme.
Il y a 1348 images d'entrainement de patient infltration.
Il y a 1000 images d'entrainement de patient pneumonie.
Il y a 219 images de validation de patient covid.
Il y a 930 images de validation de patient sain.
Il y a 102 images de validation de patient atelectasis.
Il y a 133 images de validation de patient effusion.
Il y a 193 images de validation de patient emphyseme.
Il y a 193 images de validation de patient infltration.
Il y a 333 images de validation de patient pneumonie.
Il y a 240 images test de patient covid.
Il y a 872 images test de patient sain.
Il y a 102 images test de patient atelectasis.
Il y a 133 images test de patient effusion.
Il y a 193 images test de patient emphyseme.
Il y a 387 images test de p

In [3]:
# Preprocessing :

# On rescale les images :

datagen = ImageDataGenerator(rescale=1./255)

# On definit la batch size :

batch_size = 32 

# On prepare les tableaux de donnees depuis les images :

train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

Found 7816 images belonging to 7 classes.
Found 2102 images belonging to 7 classes.
Found 2260 images belonging to 7 classes.


In [4]:
# On definit l'architecture du model :

model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7))
model.add(Activation('softmax'))

# On compile le modeles :

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# On affiche le model :

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        1

In [5]:
# On definit les paramétres pour l'entrainement :

epochs = 30
train_samples = 7816
validation_samples = 2102
test_samples = 2260

In [6]:
# On definit un callback

my_callbacks = [
    EarlyStopping(monitor='val_accuracy',mode='max',patience = 3,verbose=2,restore_best_weights=True)
]

In [7]:
historique = model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        callbacks = my_callbacks,
        validation_data=validation_generator,
        validation_steps=validation_samples// batch_size)

Epoch 1/30
244/244 [==============================] - 2707s 11s/step - loss: 1.4836 - accuracy: 0.3999 - val_loss: 0.7726 - val_accuracy: 0.6918
Epoch 2/30
244/244 [==============================] - 3835s 16s/step - loss: 1.1825 - accuracy: 0.4983 - val_loss: 0.6440 - val_accuracy: 0.7298
Epoch 3/30
244/244 [==============================] - 2018s 8s/step - loss: 1.0872 - accuracy: 0.5310 - val_loss: 0.6298 - val_accuracy: 0.7462
Epoch 4/30
244/244 [==============================] - 2996s 12s/step - loss: 1.0245 - accuracy: 0.5513 - val_loss: 0.6038 - val_accuracy: 0.7486
Epoch 5/30
244/244 [==============================] - 1755s 7s/step - loss: 0.9723 - accuracy: 0.5731 - val_loss: 0.5641 - val_accuracy: 0.7529
Epoch 6/30
244/244 [==============================] - 1508s 6s/step - loss: 0.9412 - accuracy: 0.5817 - val_loss: 0.6112 - val_accuracy: 0.7385
Epoch 7/30
244/244 [==============================] - 1488s 6s/step - loss: 0.9023 - accuracy: 0.5865 - val_loss: 0.5857 - val_accura

In [8]:
# Enregistrement des donnees

np.save('models/historique_scratch_224_224_MAX_G_HUGO.npy',historique.history)

model.save('models/scratchFull_MAX_G_224_224')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/scratchFull_MAX_G_224_224/assets


In [9]:
# Evaluation des modeles

resultValidation = model.evaluate_generator(validation_generator, validation_samples)
resultTest = model.evaluate_generator(test_generator, test_samples)


print(resultValidation)
print(resultTest)

[0.5843271255797993, 0.75787854]
[0.754688113149816, 0.6938483]


In [10]:
def testImage(path):
    img = load_img(path, target_size=(224,224))
    img = img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    y = model.predict(img)
    return y

In [13]:
for i in range(10):
    result = testImage(Image+"/TEST/ATELECTASIS/"+ImageTESTATELECTASIS[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    print(itemindex[0][0],end=' | ')
    
print('End ---- ')

for i in range(10):
    result = testImage(Image+"/TEST/COVID/"+ImageTESTCOVID[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    print(itemindex[0][0],end=' | ')
    
print('End ---- ')

for i in range(10):
    result = testImage(Image+"/TEST/EFFUSION/"+ImageTESTEFFUSION[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    print(itemindex[0][0],end=' | ')
    
print('End ---- ')

for i in range(10):
    result = testImage(Image+"/TEST/EMPHYSEMA/"+ImageTESTEMPHYSEMA[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    print(itemindex[0][0],end=' | ')
    
print('End ---- ')

for i in range(10):
    result = testImage(Image+"/TEST/INFILTRATION/"+ImageTESTINFILTRATION[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    print(itemindex[0][0],end=' | ')
    
print('End ---- ')

for i in range(10):
    result = testImage(Image+"/TEST/NORMAL/"+ImageTESTNORMAL[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    print(itemindex[0][0],end=' | ')
    
print('End ---- ')

for i in range(10):
    result = testImage(Image+"/TEST/PNEUMONIE/"+ImageTESTPNEUMONIE[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    print(itemindex[0][0],end=' | ')
    
print('End ---- ')

2 | 0 | 0 | 0 | 0 | 3 | 0 | 2 | 2 | 4 | End ---- 
1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | End ---- 
1 | 0 | 0 | 0 | 2 | 0 | 1 | 1 | 2 | 0 | End ---- 
2 | 3 | 2 | 0 | 1 | 2 | 0 | 2 | 0 | 4 | End ---- 
2 | 0 | 2 | 3 | 2 | 2 | 0 | 1 | 2 | 0 | End ---- 
5 | 5 | 5 | 5 | 5 | 5 | 5 | 5 | 5 | 5 | End ---- 
6 | 6 | 6 | 6 | 6 | 6 | 1 | 6 | 6 | 6 | End ---- 


In [18]:
resultTotal = 0

for i in range(230):
    result = testImage(Image+"/TEST/NORMAL/"+ImageTESTNORMAL[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    if itemindex[0][0] == 5:
        resultTotal = resultTotal + 1
    
print('End ---- ')
print(resultTotal)

End ---- 
212


In [19]:
resultTotal = 0

for i in range(333):
    result = testImage(Image+"/TEST/PNEUMONIE/"+ImageTESTPNEUMONIE[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    if itemindex[0][0] == 6:
        resultTotal = resultTotal + 1
    
print('End ---- ')
print(resultTotal)

End ---- 
291


In [20]:
resultTotal = 0

for i in range(200):
    result = testImage(Image+"/TEST/COVID/"+ImageTESTCOVID[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    if itemindex[0][0] == 1:
        resultTotal = resultTotal + 1
    
print('End ---- ')
print(resultTotal)

End ---- 
178


In [22]:
resultTotal = 0

for i in range(100):
    result = testImage(Image+"/TEST/ATELECTASIS/"+ImageTESTATELECTASIS[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    if itemindex[0][0] == 0:
        resultTotal = resultTotal + 1
    
print('End ---- ')
print(resultTotal)

End ---- 
41


In [23]:
resultTotal = 0

for i in range(100):
    result = testImage(Image+"/TEST/EFFUSION/"+ImageTESTEFFUSION[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    if itemindex[0][0] == 0:
        resultTotal = resultTotal + 1
    
print('End ---- ')
print(resultTotal)

End ---- 
27


In [25]:
resultTotal = 0

for i in range(180):
    result = testImage(Image+"/TEST/EMPHYSEMA/"+ImageTESTEMPHYSEMA[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    if itemindex[0][0] == 0:
        resultTotal = resultTotal + 1
    
print('End ---- ')
print(resultTotal)

End ---- 
44


In [26]:
resultTotal = 0

for i in range(180):
    result = testImage(Image+"/TEST/INFILTRATION/"+ImageTESTINFILTRATION[i])[0]
    item = max(result)
    itemindex = np.where(result==item)
    if itemindex[0][0] == 0:
        resultTotal = resultTotal + 1
    
print('End ---- ')
print(resultTotal)

End ---- 
39


In [ ]:
# On peut noter que les plus mauvaises performances
# semblent corrélé au maladie qui ont le moins d'image pour l'entrainement.
# Les donnees semblent donc etre le coeur du sujet.